In [ ]:
!pip install pyarrow==0.15
!pip install snappy
!pip install fastparquet
!pip install python-snappy
!sudo apt-get install libsnappy-dev

In [ ]:
! pip3 install python-snappy

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")
import os
from tqdm.notebook import tqdm

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
MAIN_FOLDER = "/content/drive/My Drive/HackLab/3_Lenta/data"

### Let's work only with St.Petersburg data

In [6]:
df_file_plants = f"{MAIN_FOLDER}/plants.csv"
df_plants = pd.read_csv(df_file_plants)
df_plants.shape

(388, 3)

In [9]:
spb.head()

,plant,plant_type,city
0,95b09698fda1f64af16708ffb859eab9,HM,St. Petersburg
1,926abae84a4bd33c834bc6b981b8cf30,HM,St. Petersburg
2,ae2bac2e4b4da805d01b2952d7e35ba4,HM,St. Petersburg
3,0e7e3cf0ded4d9db8b376b317c007f99,HM,St. Petersburg
4,540bd55a2cf295b8ea9cd78650e89d03,HM,St. Petersburg


In [12]:
spb = df_plants[df_plants.city == 'St. Petersburg']
spb = spb.plant.values

In [13]:
df_file_transactions = f"{MAIN_FOLDER}/transactions.parquet"
df_transactions = pd.read_parquet(df_file_transactions, engine='fastparquet')

In [14]:
df_transactions.head()

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,004,90fca68123e692a005a4edeadd94b2f2,232cfe8b28970434f685b737737b26d4,2.0,146.98,0
1,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,003,90fca68123e692a005a4edeadd94b2f2,12ec64451b8e426db16f3f05ea770901,1.0,249.99,1
2,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,012,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
3,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,013,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
4,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,011,90fca68123e692a005a4edeadd94b2f2,571c6d3b559db8445e05265649eb30ab,2.0,53.98,1


In [19]:
transactions = []
for m in spb:
    transactions += [df_transactions[df_transactions.plant == m]]

In [20]:
del df_transactions

In [21]:
import gc
gc.collect()

12616

In [22]:
transactions = pd.concat(transactions, 0)

In [24]:
transactions.to_csv(f"{MAIN_FOLDER}/transactions.csv", index = False)

In [3]:
df_transactions = pd.read_csv(f"{MAIN_FOLDER}/transactions.csv")

In [5]:
transactions = df_transactions[df_transactions.plant == '95b09698fda1f64af16708ffb859eab9']
transactions = transactions.drop(columns='plant')

In [6]:
transactions

,chq_id,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,6,5afb724ed4347c362a0812ab6d9fe7e5,c9ce1692b17322492207e481d80634f9,1.000,32.09,0
1,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,9,5afb724ed4347c362a0812ab6d9fe7e5,376eec543ce52c6af676b423b9914c25,1.000,339.73,1
2,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,2,5afb724ed4347c362a0812ab6d9fe7e5,20aa28101e1ea320a7fef318a54ef6bd,1.000,34.99,1
3,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,5,5afb724ed4347c362a0812ab6d9fe7e5,2d35d4239408ad30fc73ffa7c88aaf4b,1.000,62.99,0
4,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,17,5afb724ed4347c362a0812ab6d9fe7e5,c99eb105e8573281e2e96351185963b0,1.000,69.99,1
...,...,...,...,...,...,...,...,...
442184,96cb1c05aca906788c7b785c420b50b7,2016-10-31,4,5253747ad7dabb4d71e4e27045841fc9,8dcea256fe53d32358007bc19fa999c8,1.000,89.71,1
442185,1aae1aee6ca13e8db0107fdb81e8015e,2016-10-31,1,5253747ad7dabb4d71e4e27045841fc9,991177ba44a4cb427de72d478996cbb8,0.118,23.60,0
442186,96cb1c05aca906788c7b785c420b50b7,2016-10-31,5,5253747ad7dabb4d71e4e27045841fc9,a71fe399ea961fa3a82168e3eb8b0246,1.000,24.79,0
442187,96cb1c05aca906788c7b785c420b50b7,2016-10-31,3,5253747ad7dabb4d71e4e27045841fc9,8305182f888a11fc7fcf70ef9c377b76,1.000,39.99,1


In [7]:
transactions.to_csv(f"{MAIN_FOLDER}/transactions_1hm.csv", index = False)

### Label data

If a client has attended the market less than $5$ times then he is unlikely a regular customer. And we have a hypothesis that if transactions of a client did not occur last $2$ months then this client most probably left.

In [30]:
df_transactions = pd.read_csv(f"{MAIN_FOLDER}/transactions_1hm.csv")

In [31]:
df_transactions.head()

,chq_id,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,6,5afb724ed4347c362a0812ab6d9fe7e5,c9ce1692b17322492207e481d80634f9,1.0,32.09,0
1,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,9,5afb724ed4347c362a0812ab6d9fe7e5,376eec543ce52c6af676b423b9914c25,1.0,339.73,1
2,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,2,5afb724ed4347c362a0812ab6d9fe7e5,20aa28101e1ea320a7fef318a54ef6bd,1.0,34.99,1
3,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,5,5afb724ed4347c362a0812ab6d9fe7e5,2d35d4239408ad30fc73ffa7c88aaf4b,1.0,62.99,0
4,015eb8bbd35e0d2950e980ead79b30ad,2016-10-28,17,5afb724ed4347c362a0812ab6d9fe7e5,c99eb105e8573281e2e96351185963b0,1.0,69.99,1


In [32]:
cl_pd = pd.DataFrame(columns=['client_id','vis_num','first','last'])
cl_pd['client_id'] = df_transactions.client_id.unique()

In [34]:
for cl in cl_pd.client_id:
      cl_pd.vis_num[cl_pd.client_id == cl] = df_transactions.client_id[df_transactions.client_id == cl].count()

cl_pd

,client_id,vis_num,first,last
0,5afb724ed4347c362a0812ab6d9fe7e5,1607,NaN,NaN
1,94ddced06e5c60f3e9e6cd15fd1ff629,83,NaN,NaN
2,ad7a8ac337c13377529856e3c28c8c69,71,NaN,NaN
3,0a09e7a042aa04f4bd6083996399ba92,2053,NaN,NaN
4,ddc9ad154489e3960f4b19ca268c6daf,567,NaN,NaN
...,...,...,...,...
2136,3e056b51893975c7896fa3e38e6d0e71,20,NaN,NaN
2137,d7ca6d0ef4639987f96497e14abce108,8,NaN,NaN
2138,4e3b462951b6b6331cf1f3d3f9427ddf,10,NaN,NaN
2139,51c7c9c8c94be3be6e7a28f6122b3e56,1,NaN,NaN


In [ ]:
i = 0
for cl in cl_pd.client_id:
      cl_pd['first'][i] = df.chq_date[df_transactions.client_id == cl].min() #np.array(df[df.client_id == cl].sort_values('chq_date').chq_date)[0] 
      cl_pd['last'][i] = df.chq_date[df_transactions.client_id == cl].max()
      i+=1
cl_pd

In [12]:
cl_pd = df_transactions[['chq_date', 'client_id']].groupby('client_id')

In [35]:
client_pd = df_transactions[['chq_date', 'client_id']].groupby('client_id').agg({'chq_date': ['count', 'min', 'max']})


In [15]:
out_clients = client_pd[client_pd.chq_date['count'] <= 5].index.values  #should not be considered
left_clients = client_pd[(client_pd.chq_date['max']<'2017-08-01')&(client_pd.chq_date['count'] > 5)].index.values # label is 1, otherwise 0

### Form dataset

In [16]:
new_df = df_transactions.copy()

In [17]:
new_df['to_drop'] = new_df.client_id.apply(lambda x: x in out_clients)

In [18]:
new_df = new_df[new_df['to_drop'] == False].copy()

In [19]:
new_df.drop(columns=['to_drop'], inplace=True)

In [20]:
df_file_materials = f"{MAIN_FOLDER}/materials.csv"
df_materials = pd.read_csv(df_file_materials, index_col='material')

In [21]:
new_df = new_df.join(df_materials, on='material')

In [22]:
new_df.dropna(inplace=True)

In [23]:
mappings = {}
columns = ['client_id', 'hier_level_1', 'hier_level_2', 'hier_level_3', 'hier_level_4', 'chq_id']
for c in columns:
    mappings[c] = dict(zip(new_df[c].unique(),\
                           np.arange(len(new_df[c].unique()))))

In [25]:
rev_mappings = {}
for key in mappings.keys():
    rev_mappings[key] = {v : k for (k, v) in mappings[key].items()}

In [24]:
hier_level1 = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_1.nunique()))
hier_level2 = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_2.nunique()))
hier_level3 = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_3.nunique()))
hier_level4 = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_4.nunique()))
hier_level1_sm = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_1.nunique()))
hier_level2_sm = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_2.nunique()))
hier_level3_sm = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_3.nunique()))
hier_level4_sm = np.zeros((new_df.chq_id.nunique(), new_df.hier_level_4.nunique()))
chq_client = np.zeros(new_df.chq_id.nunique(),)

for i, row in tqdm(new_df.iterrows()):
    hier_level1[mappings['chq_id'][row.chq_id], 
                mappings['hier_level_1'][row.hier_level_1]] += row.sales_count
    hier_level1_sm[mappings['chq_id'][row.chq_id], 
                   mappings['hier_level_1'][row.hier_level_1]] += row.sales_sum
    hier_level2[mappings['chq_id'][row.chq_id], 
                mappings['hier_level_2'][row.hier_level_2]] += row.sales_count
    hier_level2_sm[mappings['chq_id'][row.chq_id], 
                   mappings['hier_level_2'][row.hier_level_2]] += row.sales_sum
    hier_level3[mappings['chq_id'][row.chq_id], 
                mappings['hier_level_3'][row.hier_level_3]] += row.sales_count
    hier_level3_sm[mappings['chq_id'][row.chq_id], 
                   mappings['hier_level_3'][row.hier_level_3]] += row.sales_sum
    hier_level4[mappings['chq_id'][row.chq_id], 
                mappings['hier_level_4'][row.hier_level_4]] += row.sales_count
    hier_level4_sm[mappings['chq_id'][row.chq_id], 
                   mappings['hier_level_4'][row.hier_level_4]] += row.sales_sum
    chq_client[mappings['chq_id'][row.chq_id]] = mappings['client_id'][row.client_id]

In [27]:
hier_level_1 = np.zeros((new_df.client_id.nunique(), new_df.hier_level_1.nunique()))
hier_level_2 = np.zeros((new_df.client_id.nunique(), new_df.hier_level_2.nunique()))
hier_level_3 = np.zeros((new_df.client_id.nunique(), new_df.hier_level_3.nunique()))
hier_level_4 = np.zeros((new_df.client_id.nunique(), new_df.hier_level_4.nunique()))
hier_level_1_sm = np.zeros((new_df.client_id.nunique(), new_df.hier_level_1.nunique()))
hier_level_2_sm = np.zeros((new_df.client_id.nunique(), new_df.hier_level_2.nunique()))
hier_level_3_sm = np.zeros((new_df.client_id.nunique(), new_df.hier_level_3.nunique()))
hier_level_4_sm = np.zeros((new_df.client_id.nunique(), new_df.hier_level_4.nunique()))
counts = np.zeros(new_df.client_id.nunique())

for i in tqdm(range(len(mappings['client_id']))):
    hier_level_1[i] = hier_level1[chq_client == i].sum(0)
    hier_level_1_sm[i] = hier_level1_sm[chq_client == i].sum(0)
    hier_level_2[i] = hier_level2[chq_client == i].sum(0)
    hier_level_2_sm[i] = hier_level2_sm[chq_client == i].sum(0)
    hier_level_3[i] = hier_level3[chq_client == i].sum(0)
    hier_level_3_sm[i] = hier_level3_sm[chq_client == i].sum(0)
    hier_level_4[i] = hier_level4[chq_client == i].sum(0)
    hier_level_4_sm[i] = hier_level4_sm[chq_client == i].sum(0)
    counts[i] = (chq_client == i).sum(0)

In [28]:
hier_level_1 /= counts[:, None]
hier_level_1_sm /= counts[:, None]
hier_level_2 /= counts[:, None]
hier_level_2_sm /= counts[:, None]
hier_level_3 /= counts[:, None]
hier_level_3_sm /= counts[:, None]
hier_level_4 /= counts[:, None]
hier_level_4_sm /= counts[:, None]

In [39]:
hier_features = np.concatenate([hier_level_1, hier_level_2, hier_level_3, hier_level_4], axis=1)

In [40]:
dataset_df = pd.DataFrame(hier_features, columns=[f'fier_feat{i}' for i in range(hier_features.shape[1])])
dataset_df['client_id'] = [rev_mappings['client_id'][i] for i in range(hier_features.shape[0])]

In [41]:
count = new_df[['chq_id', 'sales_count', 'sales_sum']].groupby(['chq_id']).sum()

In [42]:
count['chq_id'] = count.index
count.index = np.arange(len(count))
count.head()

,sales_count,sales_sum,chq_id
0,55.143,3266.0,000145ef3b683a9835b83dfdb19cf8cf
1,35.062,5840.0,00044e2d1a916a214d04b035ea2d72bc
2,35.648,1914.0,0006560dc613f7bb5aa151c335bd1af0
3,7.418,352.0,0008b5b61e0bd640fbf159424bea48fd
4,7.000,276.0,000f12d39823b063034adfd82331f770


In [43]:
count['client_id'] = count.chq_id.apply(lambda x: rev_mappings['client_id'][int(chq_client[mappings['chq_id'][x]])])

In [44]:
dataset_df['max_count'] = None
dataset_df['avg_count'] = None
dataset_df['max_sum'] = None 
dataset_df['avg_sum'] = None

In [45]:
for i, cl in tqdm(enumerate(dataset_df.client_id)):
      dataset_df['max_count'][i] = count.sales_count[count.client_id == cl].max()
      dataset_df['avg_count'][i] = count.sales_count[count.client_id == cl].mean()
      dataset_df['max_sum'][i] = count.sales_sum[count.client_id == cl].max() 
      dataset_df['avg_sum'][i] = count.sales_sum[count.client_id == cl].mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [46]:
dataset_df.head()

,fier_feat0,fier_feat1,fier_feat2,fier_feat3,fier_feat4,fier_feat5,fier_feat6,fier_feat7,fier_feat8,fier_feat9,fier_feat10,fier_feat11,fier_feat12,fier_feat13,fier_feat14,fier_feat15,fier_feat16,fier_feat17,fier_feat18,fier_feat19,fier_feat20,fier_feat21,fier_feat22,fier_feat23,fier_feat24,fier_feat25,fier_feat26,fier_feat27,fier_feat28,fier_feat29,fier_feat30,fier_feat31,fier_feat32,fier_feat33,fier_feat34,fier_feat35,fier_feat36,fier_feat37,fier_feat38,fier_feat39,...,fier_feat2189,fier_feat2190,fier_feat2191,fier_feat2192,fier_feat2193,fier_feat2194,fier_feat2195,fier_feat2196,fier_feat2197,fier_feat2198,fier_feat2199,fier_feat2200,fier_feat2201,fier_feat2202,fier_feat2203,fier_feat2204,fier_feat2205,fier_feat2206,fier_feat2207,fier_feat2208,fier_feat2209,fier_feat2210,fier_feat2211,fier_feat2212,fier_feat2213,fier_feat2214,fier_feat2215,fier_feat2216,fier_feat2217,fier_feat2218,fier_feat2219,fier_feat2220,fier_feat2221,fier_feat2222,fier_feat2223,client_id,max_count,avg_count,max_sum,avg_sum
0,12.798262,0.901639,1.319672,0.672131,0.139344,1.491803,1.229508,1.022197,0.426230,0.573770,0.885246,0.553230,0.853254,0.000000,0.131148,0.246426,0.302213,0.008197,0.418033,0.397844,0.491803,0.081967,0.303279,0.040984,0.040984,0.090164,0.07377,0.131148,0.00,0.000,0.000000,0.073770,0.032787,0.000000,0.418033,0.02459,0.188525,0.377049,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5afb724ed4347c362a0812ab6d9fe7e5,31.078,13.6999,9639,1375.36
1,7.122375,5.625000,0.250000,0.000000,0.000000,1.000000,0.875000,1.980250,0.000000,0.250000,0.000000,0.000000,0.000000,0.125000,0.000000,0.159000,1.875000,0.125000,1.750000,0.086250,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.25,0.375,0.000000,0.125000,0.000000,0.000000,0.000000,0.00000,2.375000,0.000000,0.375000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,94ddced06e5c60f3e9e6cd15fd1ff629,24.137,12.7474,2523,1109.5
2,7.064125,2.500000,0.250000,0.125000,0.000000,0.000000,0.500000,0.091250,0.500000,0.000000,0.750000,0.385250,0.247375,0.000000,0.875000,1.785875,1.250000,0.125000,1.000000,0.109750,0.125000,0.000000,0.000000,0.125000,0.000000,0.125000,0.00000,0.500000,0.00,0.125,0.000000,0.375000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,ad7a8ac337c13377529856e3c28c8c69,14.187,9.56412,1577,1208.38
3,53.435983,20.900000,1.283333,0.133333,0.100000,1.116667,0.833333,4.799683,1.833333,13.783333,4.550000,0.685700,0.630400,0.133333,5.450000,0.930383,1.781150,0.150000,6.283333,1.438083,0.750000,3.700000,1.183333,0.766667,0.566667,0.450000,0.00000,1.233333,0.20,0.650,0.033333,1.000000,0.050000,1.650000,0.739600,0.45000,2.866667,0.316667,0.133333,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,0a09e7a042aa04f4bd6083996399ba92,178.878,74.336,20631,8924.9
4,6.015956,3.055556,0.700000,0.000000,0.233333,0.222222,0.155556,1.274411,0.711111,0.022222,0.488889,0.134833,0.145822,0.211111,0.366667,0.052022,0.311111,0.033333,1.200000,0.163844,0.011111,0.288889,0.333333,0.000000,0.000000,0.011111,0.00000,0.055556,0.00,0.000,0.000000,0.255556,0.144444,0.044444,0.033333,0.00000,0.000000,0.022222,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,ddc9ad154489e3960f4b19ca268c6daf,58,9.07151,7139,707.444


In [47]:
final_df = dataset_df.drop(columns=['client_id'])

In [48]:
final_df.insert(0, "client_id", dataset_df.client_id)

In [49]:
final_df['label'] = final_df.client_id.apply(lambda x: int(x in left_clients))

In [50]:
final_df.head()

,client_id,fier_feat0,fier_feat1,fier_feat2,fier_feat3,fier_feat4,fier_feat5,fier_feat6,fier_feat7,fier_feat8,fier_feat9,fier_feat10,fier_feat11,fier_feat12,fier_feat13,fier_feat14,fier_feat15,fier_feat16,fier_feat17,fier_feat18,fier_feat19,fier_feat20,fier_feat21,fier_feat22,fier_feat23,fier_feat24,fier_feat25,fier_feat26,fier_feat27,fier_feat28,fier_feat29,fier_feat30,fier_feat31,fier_feat32,fier_feat33,fier_feat34,fier_feat35,fier_feat36,fier_feat37,fier_feat38,...,fier_feat2189,fier_feat2190,fier_feat2191,fier_feat2192,fier_feat2193,fier_feat2194,fier_feat2195,fier_feat2196,fier_feat2197,fier_feat2198,fier_feat2199,fier_feat2200,fier_feat2201,fier_feat2202,fier_feat2203,fier_feat2204,fier_feat2205,fier_feat2206,fier_feat2207,fier_feat2208,fier_feat2209,fier_feat2210,fier_feat2211,fier_feat2212,fier_feat2213,fier_feat2214,fier_feat2215,fier_feat2216,fier_feat2217,fier_feat2218,fier_feat2219,fier_feat2220,fier_feat2221,fier_feat2222,fier_feat2223,max_count,avg_count,max_sum,avg_sum,label
0,5afb724ed4347c362a0812ab6d9fe7e5,12.798262,0.901639,1.319672,0.672131,0.139344,1.491803,1.229508,1.022197,0.426230,0.573770,0.885246,0.553230,0.853254,0.000000,0.131148,0.246426,0.302213,0.008197,0.418033,0.397844,0.491803,0.081967,0.303279,0.040984,0.040984,0.090164,0.07377,0.131148,0.00,0.000,0.000000,0.073770,0.032787,0.000000,0.418033,0.02459,0.188525,0.377049,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,31.078,13.6999,9639,1375.36,0
1,94ddced06e5c60f3e9e6cd15fd1ff629,7.122375,5.625000,0.250000,0.000000,0.000000,1.000000,0.875000,1.980250,0.000000,0.250000,0.000000,0.000000,0.000000,0.125000,0.000000,0.159000,1.875000,0.125000,1.750000,0.086250,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.25,0.375,0.000000,0.125000,0.000000,0.000000,0.000000,0.00000,2.375000,0.000000,0.375000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,24.137,12.7474,2523,1109.5,1
2,ad7a8ac337c13377529856e3c28c8c69,7.064125,2.500000,0.250000,0.125000,0.000000,0.000000,0.500000,0.091250,0.500000,0.000000,0.750000,0.385250,0.247375,0.000000,0.875000,1.785875,1.250000,0.125000,1.000000,0.109750,0.125000,0.000000,0.000000,0.125000,0.000000,0.125000,0.00000,0.500000,0.00,0.125,0.000000,0.375000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,14.187,9.56412,1577,1208.38,1
3,0a09e7a042aa04f4bd6083996399ba92,53.435983,20.900000,1.283333,0.133333,0.100000,1.116667,0.833333,4.799683,1.833333,13.783333,4.550000,0.685700,0.630400,0.133333,5.450000,0.930383,1.781150,0.150000,6.283333,1.438083,0.750000,3.700000,1.183333,0.766667,0.566667,0.450000,0.00000,1.233333,0.20,0.650,0.033333,1.000000,0.050000,1.650000,0.739600,0.45000,2.866667,0.316667,0.133333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016667,0.0,0.0,0.0,0.0,178.878,74.336,20631,8924.9,0
4,ddc9ad154489e3960f4b19ca268c6daf,6.015956,3.055556,0.700000,0.000000,0.233333,0.222222,0.155556,1.274411,0.711111,0.022222,0.488889,0.134833,0.145822,0.211111,0.366667,0.052022,0.311111,0.033333,1.200000,0.163844,0.011111,0.288889,0.333333,0.000000,0.000000,0.011111,0.00000,0.055556,0.00,0.000,0.000000,0.255556,0.144444,0.044444,0.033333,0.00000,0.000000,0.022222,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,58,9.07151,7139,707.444,1


In [ ]:
final_df.to_csv(f"{MAIN_FOLDER}/feat_lab.csv", index=False)